In [1]:
# Add source directories to search path
import sys
src_paths = ["macros\\"]
for src_path in src_paths:
    if not src_path in sys.path:
        sys.path.append(src_path)


In [4]:
# import dnad

# print(dnad.elemental_purity(is_pure = True, test_coverage = False))

str(False).lower_case()

AttributeError: 'str' object has no attribute 'lower_case'